In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate, get_compile_kwargs
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-10-20 08:51:21.601615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 08:51:22.150167: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [7]:
data_path = Path("data/sat1/split_stage_data_unprocessed_100hz.nc")

data = xr.load_dataset(data_path)

In [8]:
data_path = Path("data/sat1/split_stage_data_weighted_mean.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/stage_data.nc")

data = xr.load_dataset(data_path)

In [1]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

NameError: name 'split_data_on_participants' is not defined

In [4]:
model = SAT1LSTM(len(data.channels), len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

2023-10-19 16:46:53.731024: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-19 16:46:53.760552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-19 16:46:53.760714: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-19 16:46:53.762505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-19 16:46:53.762578: I tensorflow/compile

In [5]:
results = k_fold_cross_validate(
    data,
    model,
    5,
    normalization_fn=norm_dummy,
    train_kwargs={"logs_path": Path("logs/rnn_performance"), "additional_name": "LSTM"},
)
print_results(results)

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/20


2023-10-19 16:47:09.574856: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_40/output/_23'
2023-10-19 16:47:10.614493: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-19 16:47:10.700786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-19 16:47:10.9502

997/997 [==============================] - 24s 19ms/step - loss: 5.0982 - accuracy: 0.5977 - val_loss: 0.7599 - val_accuracy: 0.7133
Epoch 2/20
997/997 [==============================] - 18s 18ms/step - loss: 3.0389 - accuracy: 0.7666 - val_loss: 0.6398 - val_accuracy: 0.7721
Epoch 3/20
997/997 [==============================] - 18s 18ms/step - loss: 2.5429 - accuracy: 0.8008 - val_loss: 0.5297 - val_accuracy: 0.8059
Epoch 4/20
997/997 [==============================] - 18s 18ms/step - loss: 2.2976 - accuracy: 0.8245 - val_loss: 0.6301 - val_accuracy: 0.7729
Epoch 5/20
997/997 [==============================] - 18s 18ms/step - loss: 2.1559 - accuracy: 0.8362 - val_loss: 0.5227 - val_accuracy: 0.8014
Epoch 6/20
997/997 [==============================] - 18s 18ms/step - loss: 1.9998 - accuracy: 0.8495 - val_loss: 0.4943 - val_accuracy: 0.8199
Epoch 7/20
997/997 [==============================] - 18s 18ms/step - loss: 1.9522 - accuracy: 0.8493 - val_loss: 0.5090 - val_accuracy: 0.8120
Epo

In [12]:
model = SAT1GRU(len(data.channels), len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

In [13]:
results = k_fold_cross_validate(
    data,
    model,
    5,
    normalization_fn=norm_dummy,
    train_kwargs={"logs_path": Path("logs/rnn_performance"), "additional_name": "GRU"},
)
print_results(results)

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/20
997/997 [==============================] - 22s 19ms/step - loss: 4.9468 - accuracy: 0.6175 - val_loss: 0.6330 - val_accuracy: 0.7692
Epoch 2/20
997/997 [==============================] - 18s 18ms/step - loss: 3.2070 - accuracy: 0.7583 - val_loss: 0.5532 - val_accuracy: 0.7998
Epoch 3/20
997/997 [==============================] - 18s 18ms/step - loss: 2.3790 - accuracy: 0.8205 - val_loss: 0.5010 - val_accuracy: 0.8227
Epoch 4/20
997/997 [==============================] - 18s 18ms/step - loss: 2.1031 - accuracy: 0.8378 - val_loss: 0.4961 - val_accuracy: 0.8110
Epoch 5/20
997/997 [==============================] - 18s 18ms/step - loss: 1.9533 - accuracy: 0.8501 - val_loss: 0.4834 - val_accuracy: 0.8294
Epoch 6/20
997/997 [==============================] - 18s 18ms/step - loss: 1.8265 - accuracy: 0.8586 - val_loss: 0.4482 - val_accuracy: 0.8411
Epoch 7/20
997/997 [==============================] - 18s 18ms/step - loss: 1.79

In [6]:
model = SAT1seq2seqGRU(len(data.channels), len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

In [ ]:
train_and_evaluate(
    model, train_data, val_data, test_data, epochs=20, logs_path=Path("logs/")
)

In [ ]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/"),
    generator=SequentialSAT1DataGenerator,
)

In [11]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [6]:
model.save("models/gru")

INFO:tensorflow:Assets written to: models/gru/assets


INFO:tensorflow:Assets written to: models/gru/assets


In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 154, 30)]         0         
                                                                 
 masking (Masking)           (None, 154, 30)           0         
                                                                 
 gru (GRU)                   (None, 154, 64)           18432     
                                                                 
 gru_1 (GRU)                 (None, 154, 32)           9408      
                                                                 
 dense (Dense)               (None, 154, 5)            165       
                                                                 
Total params: 28005 (109.39 KB)
Trainable params: 28005 (109.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
